# Demonstration Thanzi La Onse notebook

This outlines ways to use the modelling framework interactively.

## A simple simulation

In [2]:
# Import general Python modules
import numpy as np
import pandas as pd

# Import the classes we need from the tlo framework
from tlo import Date, DateOffset, Person, Simulation
from tlo.test import random_birth, random_death
from tlo.util import show_changes

In [3]:
# Create a simulation with desired modules
sim = Simulation(start_date=Date(1950, 1, 1))

rb = random_birth.RandomBirth(name='rb')
rb.pregnancy_probability = 0.1
rd = random_death.RandomDeath(name='rd')
rd.death_probability = 0.01
sim.register(rb, rd)

In [4]:
# Repeatability
sim.seed_rngs(4)

In [5]:
# Set up initial population
sim.make_initial_population(n=5)

In [6]:
# Store initial population state for comparison
initial_state = sim.population.props.copy()

In [7]:
# Run simulation
sim.simulate(end_date=Date(1951, 1, 1))

In [8]:
# Compare final & initial states
final_state = sim.population.props
show_changes(sim, initial_state, final_state)

,is_pregnant,date_of_birth,children,is_alive,date_of_death
0,True,1943-11-30 00:00:00,[],True,NaT
1,True,1944-08-31 00:00:00,[],False,1950-05-01 00:00:00
2,False,1945-10-31 00:00:00,[],False,1950-03-01 00:00:00
3,False,1940-02-29 00:00:00,[],True,NaT
4,True,1948-09-30 00:00:00,[5],True,NaT
5,True,1950-11-01 00:00:00,[],True,NaT


In [9]:
# The simple display above doesn't handle 'object' type data well, e.g. the children list
for i, l in enumerate(sim.population.children):
    print('Children of person {}: {}'.format(i, l))

Children of person 0: []
Children of person 1: []
Children of person 2: []
Children of person 3: []
Children of person 4: [5]
Children of person 5: []


## Testing a 'method' in isolation

In [10]:
from tlo.methods import depression

In [11]:
# Set up a minimal simulation with our method of interest
sim = Simulation(start_date=Date(1950, 1, 1))
sim.register(depression.Depression())
sim.seed_rngs(123)
sim.make_initial_population(n=5)

In [12]:
# Add in extra properties that our module needs, which would normally be provided by other methods
from tlo import Types
pop = sim.population
pop.make_test_property('is_pregnant', Types.BOOL)
pop.make_test_property('date_of_birth', Types.DATE)
pop.make_test_property('female', Types.BOOL)
pop.make_test_property('has_hyptension', Types.BOOL)
pop.make_test_property('has_chronic_back_pain', Types.BOOL)

In [13]:
# Set up some initial values for these in the population
rng = sim.modules['Depression'].rng
pop.female = rng.choice([True, False], replace=True, size=len(pop))
pop.has_hyptension = rng.choice([True, False], replace=True, size=len(pop), p=[0.1, 0.9])
pop.has_chronic_back_pain = rng.choice([True, False], replace=True, size=len(pop), p=[0.1, 0.9])
pop.date_of_birth = pop.date_of_birth.apply(
    lambda _: Date(rng.randint(1950, 2010), rng.randint(1, 12), rng.randint(1, 28)))

In [14]:
# Create and fire our event of interest, showing changes it makes
event = depression.DepressionEvent(sim.modules['Depression'])
initial_state = sim.population.props.copy()
sim.fire_single_event(event, Date(1950, 2, 1))
final_state = sim.population.props
show_changes(sim, initial_state, final_state)

,is_depressed,date_init_depression,date_depression_resolved,ever_depressed,prob_3m_resol_depression,is_pregnant,date_of_birth,female,has_hyptension,has_chronic_back_pain
person,,,,,,,,,,
0,False,NaT,NaT,False,nan,False,1964-05-01 00:00:00,True,False,False
1,False,NaT,NaT,False,nan,False,1966-05-18 00:00:00,False,False,False
2,False,NaT,NaT,False,nan,False,2005-04-03 00:00:00,True,False,False
3,False,NaT,NaT,False,nan,False,1970-08-03 00:00:00,True,False,False
4,False,NaT,NaT,False,nan,False,1970-09-17 00:00:00,True,False,False
